In [ ]:
# Data Racing competition

In [ ]:
# Tasks

2022.06.01 
1. Standardisation of data - logarithm of numeric data
2. Outlier detection
3. Setting up the Colab Environment

In [ ]:
# Tags
regression, xgboost, catboost, lightgbm

# Reference
[xgboost basics](https://www.kaggle.com/code/carlmcbrideellis/an-introduction-to-xgboost-regression/notebook)

In [ ]:
# imports
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from numpy.random import normal
from numpy import hstack
from statsmodels.distributions.empirical_distribution import ECDF

# utils
from utils import tab_data

In [ ]:
# settings
import warnings
warnings.filterwarnings('ignore')
#print all rows of a df in ipython shell 
pd.set_option('display.max_rows', None)
#print all columns of a df in ipython shell 
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.set_option("display.precision", 6)


# optional
pd.set_option('display.max_columns',100)
sns.set_style('darkgrid')

In [ ]:
aktelh - aktivált időbeli elhatárolás
arbevert - árbevétel elért
arbevexp - árbevétel export
befpue - befektetett pénzügyi eszközök
begyeb - befektetett egyéb
celtart - céltartalék
eredadel - eredeti adózás előtt
eredadoz - eredeti adózott
eredpenz - eredeti pénz
eredtar - eredeti tartozás
ereduzem - eredeti uzemeltetés
ertpapir - értékpapir
hoskot01 - hosszúlejáratú kötelezettségek
hoskot - hosszulejáratú kötelezettségek
immat - immateriális javak
jetok - jegyzett tőke
kecs - keszlet értékcsökkenés
keszl - készlet
kovet - követelések 
letszam - létszám
merlfoo - mérleg főösszeg
penzeszk - pénzeszközök
ranyag - ráfordítások anyag
regyeb - ráforditások egyéb
rovkot_01 -
rovkot - rövid lejáratú kötelezettségek
rszem - ráforditás személyzet
sajattok - saját tőke
targyie - tárgyi eszköz

# Data Enginieering

In [ ]:
# 1. load data
print("1. Load data")

# path = 'e:\PycharmProjects\CompleteEDA'
path = os.path.abspath(os.getcwd())
train = 'train.csv'
test = 'test.csv'

path_to_train = os.path.join(path, 'data', train)

path_to_test = os.path.join(path, 'data', test)

df_train = pd.read_csv(path_to_train)
df_test_orig = pd.read_csv(path_to_test)

In [ ]:
# 2. create df for TRAINING
print("2. Create df for TRAINING")
df = df_train.copy()

# 3. transform nas and erroneous big alakul_ev to 2013 
print("3. Transform nas and erroneous big alakul_ev to 2013")

col = 'alakul_ev'
df[col] = np.where((df.alakul_ev.isna()) |  (df.alakul_ev > 2016), 2013, df.alakul_ev)

# 4. fill missing categorical values with the most frequent value
print("4. Fill missing categorical values with the most frequent value")

from sklearn.impute import SimpleImputer

# get columns with missing values
columns_cat_missing_values = [column for column in df.columns if df[column].isna().sum() > 0]

# fill with most frequent values
for col in columns_cat_missing_values:
    imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
    df[col] = imp.fit_transform(df[col].values.reshape(-1, 1))

# 5. get numeric and object columns per years and divide numeric float ones by letszam
print("5. Get numeric and object columns per years and divide numeric float ones by letszam")

cols_no_year = ['id', 'target_reg', 'alakul_ev']

numeric_cols_2014 = [column for column in df.select_dtypes(exclude=['int64'], include=['float64']).columns if "2014" in column]
numeric_cols_2014 = [item for item in numeric_cols_2014 if item not in  ['KATEGORIKUS_VALTOZO_2014', 'letszam_2014']]
object_cols_2014 = [column for column in df.select_dtypes( include=['object']).columns if "2014" in column]

numeric_cols_2015 = [column for column in df.select_dtypes(exclude=['int64'], include=['float64']).columns if "2015" in column]
numeric_cols_2015 = [item for item in numeric_cols_2015 if item not in  ['KATEGORIKUS_VALTOZO_2015', 'letszam_2015']]
object_cols_2015 = [column for column in df.select_dtypes(include=['object']).columns if "2015" in column]

numeric_cols_2016 = [column for column in df.select_dtypes(exclude=['int64'], include=['float64']).columns if "2016" in column]
numeric_cols_2016 = [item for item in numeric_cols_2016 if item not in  ['KATEGORIKUS_VALTOZO_2016', 'letszam_2016']]
object_cols_2016 = [column for column in df.select_dtypes(include=['object']).columns if "2016" in column]


# divide all numeric features by letszam expect of KATEGORIKUS_VALTOZO_x and letszam_x by year
for col in numeric_cols_2014:
    df[col] = np.where(df.letszam_2014!=0, df[col]/df.letszam_2014,0)
    #df[col] = np.log(df[col])
for col in numeric_cols_2015:
    df[col] = np.where(df.letszam_2015!=0, df[col]/df.letszam_2015,0)
    #df[col] = np.log(df[col])
for col in numeric_cols_2016:
    df[col] = np.where(df.letszam_2016!=0, df[col]/df.letszam_2016,0)
    #df[col] = np.log(df[col])

# divide target_reg by letszam_2016
col = 'target_reg'
df[col] = np.where(df.letszam_2016!=0, df[col]/df.letszam_2016,0)

# 6. extract the categorical columns to add into the model 
print("6. Extract the categorical columns to add into the model")

cols_no_year = ['id', 'target_reg', 'alakul_ev']
cols_kat_valt = ['KATEGORIKUS_VALTOZO_2014', 'KATEGORIKUS_VALTOZO_2015', 'KATEGORIKUS_VALTOZO_2016']

columns_obj = [column for column in df.select_dtypes(include='object').columns if ("2014"  in column )|("2015"  in column )|("2016"  in column )]

# 7. transform categorical columns into percentage ones
print("7. Transform categorical columns into percentage ones")

for col in columns_obj:
    # calculate the percentage 
    df_grouped = df[col].value_counts(normalize = True)
    # get the index as column
    df_grouped = df_grouped.reset_index()
    # change the column names :: orig column and percent
    percent_col = '%'+col
    df_grouped.columns = [col, percent_col]
    # merge the percent values to orig df
    df = pd.merge(df, df_grouped, how='left')

# 8. calculate percentage by count on alakul_ev
print("8. Calculate percentage by count on alakul_ev")

col = 'alakul_ev'

# calculate the percentage 
df_grouped = df[col].value_counts(normalize = True)
# get the index as column
df_grouped = df_grouped.reset_index()
# change the column names :: orig column and percent
percent_col = '%'+col
df_grouped.columns = [col, percent_col]
# merge the percent values to orig df
df = pd.merge(df, df_grouped, how='left')

# 9. create columns list for percentages columns
print("9. Create columns list for percentages columns")

columns_obj_perc = [column for column in df.columns if "%" in column]

# 10. apply PowerTransformer for numeric float and percentage float columns 
print("10. Apply PowerTransformer for numeric float and percentage float columns")

from sklearn.preprocessing import PowerTransformer

# numeric colums :: divided/letszam and categorical percentage columns
numeric_cols_to_transform = numeric_cols_2014 + numeric_cols_2015 + numeric_cols_2016 + columns_obj_perc

power_features = PowerTransformer(method="yeo-johnson",  standardize=True).fit_transform(df[numeric_cols_to_transform].values)
df[numeric_cols_to_transform] = pd.DataFrame(power_features, index=df.index, columns=df[numeric_cols_to_transform].columns)

# 11. transform target_reg, saved it in transformed
print("11. Transform target_reg, saved it in transformed")

col = ['target_reg']
ptt = PowerTransformer(method="yeo-johnson",  standardize=True)
power_target_feature = ptt.fit_transform(df[col].values)
print("Lamdas for target_reg :: {}".format(ptt.lambdas_))
df['transformed'] = pd.DataFrame(power_target_feature, index=df.index, columns=df[col].columns)

# 12. model features selection
print("12. model features selection")

model_columns = ['target_reg'] + ['transformed']+ cols_kat_valt + numeric_cols_2014 + numeric_cols_2015 + numeric_cols_2016 + columns_obj_perc

df_model = df[model_columns]

In [ ]:
# 2. create df for testing
print("2. Create df for TESTING")

print("2. Create df for TRAINING")
df_test = df_test_orig.copy()

# 3. transform nas and erroneous big alakul_ev to 2013 
print("3. Transform nas and erroneous big alakul_ev to 2013")

col = 'alakul_ev'
df_test[col] = np.where((df_test.alakul_ev.isna()) |  (df_test.alakul_ev > 2016), 2013, df_test.alakul_ev)

# 4. fill missing categorical values with the most frequent value
print("4. Fill missing categorical values with the most frequent value")

from sklearn.impute import SimpleImputer

# get columns with missing values
columns_cat_missing_values = [column for column in df_test.columns if df_test[column].isna().sum() > 0]

# fill with most frequent values
for col in columns_cat_missing_values:
    imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
    df_test[col] = imp.fit_transform(df_test[col].values.reshape(-1, 1))

# 5. get numeric and object columns per years and divide numeric float ones by letszam
print("5. Get numeric and object columns per years and divide numeric float ones by letszam")

cols_no_year = ['id', 'target_reg', 'alakul_ev']

numeric_cols_2014 = [column for column in df_test.select_dtypes(exclude=['int64'], include=['float64']).columns if "2014" in column]
numeric_cols_2014 = [item for item in numeric_cols_2014 if item not in  ['KATEGORIKUS_VALTOZO_2014', 'letszam_2014']]
object_cols_2014 = [column for column in df_test.select_dtypes( include=['object']).columns if "2014" in column]

numeric_cols_2015 = [column for column in df_test.select_dtypes(exclude=['int64'], include=['float64']).columns if "2015" in column]
numeric_cols_2015 = [item for item in numeric_cols_2015 if item not in  ['KATEGORIKUS_VALTOZO_2015', 'letszam_2015']]
object_cols_2015 = [column for column in df_test.select_dtypes(include=['object']).columns if "2015" in column]

numeric_cols_2016 = [column for column in df_test.select_dtypes(exclude=['int64'], include=['float64']).columns if "2016" in column]
numeric_cols_2016 = [item for item in numeric_cols_2016 if item not in  ['KATEGORIKUS_VALTOZO_2016', 'letszam_2016']]
object_cols_2016 = [column for column in df_test.select_dtypes(include=['object']).columns if "2016" in column]


# divide all numeric features by letszam expect of KATEGORIKUS_VALTOZO_x and letszam_x by year
for col in numeric_cols_2014:
    df_test[col] = np.where(df_test.letszam_2014!=0, df_test[col]/df_test.letszam_2014,0)
    #df[col] = np.log(df[col])
for col in numeric_cols_2015:
    df_test[col] = np.where(df_test.letszam_2015!=0, df_test[col]/df_test.letszam_2015,0)
    #df[col] = np.log(df[col])
for col in numeric_cols_2016:
    df_test[col] = np.where(df_test.letszam_2016!=0, df_test[col]/df_test.letszam_2016,0)
    #df[col] = np.log(df[col])


# 6. extract the categorical columns to add into the model 
print("6. Extract the categorical columns to add into the model")

cols_no_year = ['id', 'target_reg', 'alakul_ev']
cols_kat_valt = ['KATEGORIKUS_VALTOZO_2014', 'KATEGORIKUS_VALTOZO_2015', 'KATEGORIKUS_VALTOZO_2016']

columns_obj = [column for column in df_test.select_dtypes(include='object').columns if ("2014"  in column )|("2015"  in column )|("2016"  in column )]

# 7. transform categorical columns into percentage ones
print("7. Transform categorical columns into percentage ones")

for col in columns_obj:
    # calculate the percentage 
    df_grouped = df_test[col].value_counts(normalize = True)
    # get the index as column
    df_grouped = df_grouped.reset_index()
    # change the column names :: orig column and percent
    percent_col = '%'+col
    df_grouped.columns = [col, percent_col]
    # merge the percent values to orig df
    df_test = pd.merge(df_test, df_grouped, how='left')

# 8. calculate percentage by count on alakul_ev
print("8. Calculate percentage by count on alakul_ev")

col = 'alakul_ev'

# calculate the percentage 
df_grouped = df_test[col].value_counts(normalize = True)
# get the index as column
df_grouped = df_grouped.reset_index()
# change the column names :: orig column and percent
percent_col = '%'+col
df_grouped.columns = [col, percent_col]
# merge the percent values to orig df
df_test = pd.merge(df_test, df_grouped, how='left')

# 9. create columns list for percentages columns
print("9. Create columns list for percentages columns")

columns_obj_perc = [column for column in df_test.columns if "%" in column]

# 10. apply PowerTransformer for numeric float and percentage float columns 
print("10. Apply PowerTransformer for numeric float and percentage float columns")

from sklearn.preprocessing import PowerTransformer

# numeric colums :: divided/letszam and categorical percentage columns
numeric_cols_to_transform = numeric_cols_2014 + numeric_cols_2015 + numeric_cols_2016 + columns_obj_perc

power_features = PowerTransformer(method="yeo-johnson",  standardize=True).fit_transform(df_test[numeric_cols_to_transform].values)
df_test[numeric_cols_to_transform] = pd.DataFrame(power_features, index=df_test.index, columns=df_test[numeric_cols_to_transform].columns)

# 12. model features selection
print("12. model features selection")

model_columns = cols_kat_valt + numeric_cols_2014 + numeric_cols_2015 + numeric_cols_2016 + columns_obj_perc

df_test = df_test[model_columns]

# Data preparation

In [ ]:
# load data

# path = 'e:\PycharmProjects\CompleteEDA'
path = os.path.abspath(os.getcwd())
train = 'train.csv'
test = 'test.csv'

path_to_train = os.path.join(path, 'data', train)

path_to_test = os.path.join(path, 'data', test)

df_orig = pd.read_csv(path_to_train)
df_test = pd.read_csv(path_to_test)


# Quick checks

In [ ]:
df_orig.sample(3)

In [ ]:
df_orig.columns

In [ ]:
df_orig.dtypes

# Create df for preprocessing

In [ ]:
# for simplicity
df = df_orig.copy()

# Missing Values

In [ ]:
# Print missing and unique values in tabular format
from tabulate import tabulate
def tab_data(df):
    headers = ['Column', 'Null Count', 'Unique Count']
    meta_list = []
    cols = [i for i in df.columns]
    for col in cols:
        temp = []
        temp.append(col)
        temp.append(df[col].isna().sum())
        temp.append(df[col].nunique())
        meta_list.append(temp)
    print(tabulate(meta_list, headers, tablefmt='rst'))

In [ ]:
# check for missing and unique values
tab_data(df)

In [ ]:
# Deal with alakul_ev - check for errors
df_orig[(df_orig.alakul_ev.isna()) |  (df_orig.alakul_ev > 2016)]

In [ ]:
# transform nas and erroneous big alakul_ev to 2013 
#df_ = df.copy()
col = 'alakul_ev'
df[col] = np.where((df.alakul_ev.isna()) |  (df.alakul_ev > 2016), 2013, df.alakul_ev)

In [ ]:
# quick check alakul ev
df[col].value_counts()

In [ ]:
# fill missing values with the most frequent value
from sklearn.impute import SimpleImputer

# get columns with missing values
columns_cat_missing_values = [column for column in df.columns if df[column].isna().sum() > 0]

# fill with most frequent values
for col in columns_cat_missing_values:
    imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
    df[col] = imp.fit_transform(df[col].values.reshape(-1, 1))

# Feature enginieering

Initial attempt             
1. ertek - arbevetel/letszam              
2. jövedelem - arbevétel/személyi ráforditás                

Filling the missing values                  
alakul_ev with 2013                    
categorical features with the most frequent one                    

Actual feature enginieering               
transform categorical values by transforming counts of categoricals to percentage except of KATEGORIKUS_VALTOZO_201x and letszam_201x as          
letszam was used in diving the numerical features           
divide all the numeric columns by 'letszam' and omit letszam from the features                     
apply PowerTransformer(method="yeo-johnson") for all the numeric features               

In [ ]:
# 1. get numeric and object columns per years :: miscellous
cols_no_year = ['id', 'target_reg', 'alakul_ev']

numeric_cols_2014 = [column for column in df.select_dtypes(exclude=['int64'], include=['float64']).columns if "2014" in column]
numeric_cols_2014 = [item for item in numeric_cols_2014 if item not in  ['KATEGORIKUS_VALTOZO_2014', 'letszam_2014']]
object_cols_2014 = [column for column in df.select_dtypes( include=['object']).columns if "2014" in column]

numeric_cols_2015 = [column for column in df.select_dtypes(exclude=['int64'], include=['float64']).columns if "2015" in column]
numeric_cols_2015 = [item for item in numeric_cols_2015 if item not in  ['KATEGORIKUS_VALTOZO_2015', 'letszam_2015']]
object_cols_2015 = [column for column in df.select_dtypes(include=['object']).columns if "2015" in column]

numeric_cols_2016 = [column for column in df.select_dtypes(exclude=['int64'], include=['float64']).columns if "2016" in column]
numeric_cols_2016 = [item for item in numeric_cols_2016 if item not in  ['KATEGORIKUS_VALTOZO_2016', 'letszam_2016']]
object_cols_2016 = [column for column in df.select_dtypes(include=['object']).columns if "2016" in column]


# 2. jövedelem df - arbevétel/személyi ráforditás 
#df['szemely_2014'] = np.where(df.rszem_2014!=0, df.arbevert_2014/df.rszem_2014,0)
#df['szemely_2015'] = np.where(df.rszem_2015!=0, df.arbevert_2014/df.rszem_2015,0)
#df['szemely_2016'] = np.where(df.rszem_2016!=0, df.arbevert_2014/df.rszem_2016,0)

# 3. divide all numeric features by letszam expect of KATEGORIKUS_VALTOZO_x and letszam_x by year

for col in numeric_cols_2014:
    df[col] = np.where(df.letszam_2014!=0, df[col]/df.letszam_2014,0.1)
    #df[col] = np.log(df[col])
for col in numeric_cols_2015:
    df[col] = np.where(df.letszam_2015!=0, df[col]/df.letszam_2015,0.1)
    #df[col] = np.log(df[col])
for col in numeric_cols_2016:
    df[col] = np.where(df.letszam_2016!=0, df[col]/df.letszam_2016,0.1)
    #df[col] = np.log(df[col])

# 3. divide target_reg by letszam_2016
col = 'target_reg'
df[col] = np.where(df.letszam_2016!=0, df[col]/df.letszam_2016,0.1)

In [ ]:
# quick check

df[numeric_cols_2016].sample(3)

# Outliers

In [ ]:
# checking the largest and smallest values

df[['letszam_2016', 'target_reg']].nlargest(10, "letszam_2016")

In [ ]:
df[['letszam_2016', 'target_reg']].nsmallest(10, "letszam_2016")

In [ ]:
# starnge values: target_reg < 0

df[df.target_reg < 0]

In [ ]:
# Outliers - target_reg 0 values
df_ = df[df.target_reg < 0] 
df_0 = df[df.target_reg == 0] 
df_target = df[df.target_reg > 0]
df_or_not_null = df[(df.target_reg == 0) | (df.arbevexp_2014 == 0 ) | (df.arbevexp_2015 == 0) | (df.arbevexp_2016 == 0) ]
df_all_0 = df[(df.target_reg <= 0) & (df.arbevexp_2014 <= 0 ) & (df.arbevexp_2015 <= 0) & (df.arbevexp_2016 <= 0) ]
df_not_all_0 = df[~((df.target_reg <= 0) & (df.arbevexp_2014 <= 0 ) & (df.arbevexp_2015 <= 0) & (df.arbevexp_2016 <= 0)) ]

In [ ]:
# target_reg 0 values analysis

print('NR of all elements :: {}'.format(len(df)))
print('NR of negative targets :: {}'.format(len(df_)))
print('NR of 0 targets :: {}'.format(len(df_0)))
print('NR of pozitiv targets :: {}'.format(len(df_target)))
print('NR of either 1 years export is 0 :: {}'.format(len(df_or_not_null)))
print('NR of export for all years is 0 :: {}'.format(len(df_all_0)))
print('NR of export for all years is not 0 :: {}'.format(len(df_not_all_0)))

In [ ]:
# Quick checks
columns = ['alakul_ev', 'letszam_2014','arbevert_2014','arbevexp_2014', 'letszam_2015', 'arbevexp_2015', 'letszam_2016', 'arbevexp_2016', 'target_reg' ]
df_not_all_0[columns].sort_values(by='ertek_2014', ascending=False).sample(4)

In [ ]:
len(df_not_all_0[df.target_reg ==0])

# Distribution of categorical features

In [ ]:
df.select_dtypes(include='object').columns

In [ ]:
df.describe(include='object')

In [ ]:
# visualise categorical columns
columns = columns_2014_obj
for column in columns_2014_obj:
    if df[column].nunique() < 30:
        sns.countplot(y=column, data=df)
        plt.show()

In [ ]:
# quick check of categorical columns
df.SZEKHELY_IR_SZAM_2014.value_counts().describe()

In [ ]:
# quick check of categorical columns
df.SZEKHELY_IR_SZAM_2014.value_counts().nlargest(5)

In [ ]:
# extract the categorical columns to add into the model 
#df = df_orig.copy()

cols_no_year = ['id', 'target_reg', 'alakul_ev']
cols_kat_valt = ['KATEGORIKUS_VALTOZO_2014', 'KATEGORIKUS_VALTOZO_2015', 'KATEGORIKUS_VALTOZO_2016']

columns_obj = [column for column in df.select_dtypes(include='object').columns if ("2014"  in column )|("2015"  in column )|("2016"  in column )]
print("All object columns:: \n{}".format(columns_obj)) 

In [ ]:
# preprocess categorical columns - transform them in percentage ones

for col in columns_obj:
    # calculate the percentage 
    df_grouped = df[col].value_counts(normalize = True)
    # get the index as column
    df_grouped = df_grouped.reset_index()
    # change the column names :: orig column and percent
    percent_col = '%'+col
    df_grouped.columns = [col, percent_col]
    # merge the percent values to orig df
    df = pd.merge(df, df_grouped, how='left')

In [ ]:
# quick check of percentage columns
df['%MEGYE_2016'].sample(2)

In [ ]:
# quick check of percentage columns
df.dtypes

In [ ]:
# quick check of percentage
df[['MEGYE_2014', '%MEGYE_2014']].sample(3)

In [ ]:
# percentage by count on alakul_ev
col = 'alakul_ev'

# calculate the percentage 
df_grouped = df[col].value_counts(normalize = True)
# get the index as column
df_grouped = df_grouped.reset_index()
# change the column names :: orig column and percent
percent_col = '%'+col
df_grouped.columns = [col, percent_col]
# merge the percent values to orig df
df = pd.merge(df, df_grouped, how='left')

In [ ]:
# quick check alakul ev percentage
df[['alakul_ev', '%alakul_ev']].sample(6)

In [ ]:
## Visualise target by categorical

In [ ]:
# visualise KATEGORIKUS_VALTOZO
fig, ax =plt.subplots(1,3)
sns.countplot(y='KATEGORIKUS_VALTOZO_2014', data=df, ax=ax[0])
sns.countplot(y='KATEGORIKUS_VALTOZO_2015', data=df, ax=ax[1])
sns.countplot(y='KATEGORIKUS_VALTOZO_2016', data=df, ax=ax[2])
plt.show()

In [ ]:
# Segment the target variable by categorical features

for column in df.select_dtypes(include='object'):
 if df[column].nunique() < 21:
     sns.boxplot(y=column, x='target_reg', data=data)
     plt.show()

In [ ]:
# Group numeric features by each categorical feature

for column in df.select_dtypes(include='object'):
    if df[column].nunique() == 20:
        display(df.groupby(column).mean())

In [ ]:
# create columns for percentage
columns_obj_perc = [column for column in df.columns if "%" in column]
print("Percentage columns:: \n{} ".format(columns_obj_perc))

# Distribution of numerical features


In [ ]:
# quick check
numeric_cols_2014

In [ ]:
# visualise numeric columns distribution
for col in numeric_cols_2014:
    plt.figure()
    #sns.distplot(df['ertek_2016'], hist=False, kde=True)
    sns.distplot(df[col], hist=False, kde=True)
    plt.show()
    

In [ ]:
# target_reg distribution
plt.figure()
sns.distplot(np.log(df['target_reg']), hist=False, kde=True)
plt.show()

In [ ]:
# Check which Scaler is the best
from sklearn.preprocessing import StandardScaler, RobustScaler, PowerTransformer, QuantileTransformer

robust_features = RobustScaler(quantile_range=(5, 95)).fit_transform(df[numeric_cols_2014].values)
#df[numeric_cols_2014] = pd.DataFrame(robust_features, index=df.index, columns=df[numeric_cols_2014].columns)

quantile_features = QuantileTransformer(output_distribution="normal").fit_transform(df[numeric_cols_2014].values)
#df[numeric_cols_2014] = pd.DataFrame(quantile_features, index=df.index, columns=df[numeric_cols_2014].columns)

scaled_features = StandardScaler().fit_transform(df[numeric_cols_2014].values)
#df[numeric_cols_2014] = pd.DataFrame(scaled_features, index=df.index, columns=df[numeric_cols_2014].columns)

# the best transformation method

# numeric colums :: divided/letszam and categorical percentage columns
df[numeric_cols_to_transform] = numeric_cols_2014 + numeric_cols_2015 + numeric_cols_2016 + columns_obj_perc

power_features = PowerTransformer(method="yeo-johnson",  standardize=True).fit_transform(df[numeric_cols_to_transform].values)
df[numeric_cols_to_transform] = pd.DataFrame(power_features, index=df.index, columns=df[numeric_cols_to_transform].columns)


col = ['target_reg']
ptt = PowerTransformer(method="yeo-johnson",  standardize=True)
power_target_feature = ptt.fit_transform(df[col].values)
print("Lamdas for target_reg :: {}".format(ptt.lambdas_))
df['transformed'] = pd.DataFrame(power_target_feature, index=df.index, columns=df[col].columns)

In [ ]:
print("Lamdas for target_reg :: {}".format(ptt.lambdas_))
df['transformed'] = pd.DataFrame(power_target_feature, index=df.index, columns=df[col].columns)

In [ ]:
# check for missing values
numeric_cols = col + numeric_cols_to_transform
tab_data(df[numeric_cols])

In [ ]:
# quick checks 
df_orig[numeric_cols_2014].min()

In [ ]:
# quick checks
df[numeric_cols_2014].head(2)

In [ ]:
# visualise the transformations
for col in numeric_cols_2014:
    #define plotting region (1 row, 2 columns)
    fig, axes = plt.subplots(1, 3)

    #create distplot in each subplot
    sns.distplot(df_orig[col], hist=False, kde=True, ax=axes[0])
    sns.distplot(np.log(df[col]), hist=False, kde=True, ax=axes[1])
    sns.distplot(df[col], hist=False, kde=True, ax=axes[2])
    #sns.distplot(power_features[col], hist=False, kde=True, ax=axes[3])
    plt.show()

In [ ]:
# Univariate and multivariate boxplots for different categorical features

for col in numeric_cols_2014:
    #define plotting region (1 row, 2 columns)
    fig, ax =plt.subplots(1,2)
    sns.boxplot(x=df[col], ax=ax[0])
    sns.boxplot(x='agazat_2016_fo_kategoria', y=df[col], data=df, ax=ax[1])
    plt.show()

In [ ]:
# simple distribution plots for numerical features
plt.figure()
sns.distplot(np.log(df_orig['letszam_2014']), hist=False, kde=True)
plt.show()

In [ ]:
# quick check of the transformed target_reg data
print(len(df[df['transformed'] <=0]))
print(len(df[df['transformed'] ==0]))
print(len(df[df['transformed'] >=0]))

In [ ]:
# quick check of the transformed target_reg data
df[df['transformed'] >=0].nsmallest(100, "target_reg")
df[df['transformed'] >=0].nsmallest(100, "transformed")
df[df['transformed'] >=0].nlargest(10, "transformed")

In [ ]:
# quick check of the transformed target_reg data
df[df['transformed'] <=0].sample(5)

In [ ]:
# pairlots for two numeric features defining hue
categorical_cols = ['KATEGORIKUS_VALTOZO_2016', 'agazat_2016_fo_kategoria']
plt.figure()
sns.pairplot(df[['sajattok_2016', 'transformed', 'agazat_2016_fo_kategoria']], 
             hue='agazat_2016_fo_kategoria') #hue='origin', kind='reg'
plt.show()

In [ ]:
# median, quantiles
percentages = [0.25, 0.55, 0.95]

print(df[columns].describe(include='float64').median())
df[columns].quantile(percentages)

In [ ]:
df.describe(include='float64')

In [ ]:
# calculate quantile ranges
# inter quartile range - it's the distance between 75th and 25th percentiles
def IQR(column): 
    q25, q75 = column.quantile([0.25, 0.75])
    return q75-q25

# lower_bound
def lower_bound(column):
    q25 = column.quantile(0.25)
    q75 = column.quantile(0.75)
    iqr = q75 - q25
    lower_bound = q25 -(1.5 * iqr) 
    return lower_bound

# Upper_bound
def upper_bound(column):
    q25 = column.quantile(0.25)
    q75 = column.quantile(0.75)
    iqr = q75 - q25
    upper_bound = q75 + (1.5 * iqr) 
    return upper_bound

# range - It's the difference between input's maximum and minimum values
# range() is already a built-in function in Python - another name for custom function
def range_f(column):
    return column.max() - column.min()

stats_list = [
    'min', 'max', 
    range_f, # custom function 
    'mean', 'median',  'std',
    np.var, # numpy function
    IQR,   # custom function
    lower_bound, # custom function
    upper_bound, # custom function
]

summary_stats = df[columns].agg(func=stats_list)

# custom names for the statistics - in the same order as in the 'stat_list' variable
pretty_names = [
    'Minimum', 'Maximum', 'Range', 'Mean', 'Median', 
    'Standard Deviation', 'Variance', 'IQR', 'Lower_bound', 'Upper_bound']

# update the index labels with custom names
summary_stats.index = pretty_names
summary_stats

In [ ]:
# Nr of outliers for different numeric columns

print('Nr of lower_bound outliers :: {} '.format(len(df[df.log_ertek_2016 < 7])))
print('Nr of upper_bound outliers :: {} '.format(len(df[df.log_ertek_2016 > 12])))

In [ ]:
# Univariate and multivariate boxplots for numeric columns
fig, ax =plt.subplots(1,2)
sns.boxplot(x=df['target_reg'], ax=ax[0])
sns.boxplot(x='agazat_2016_fo_kategoria', y='target_reg', data=df, ax=ax[1])
plt.show()

In [ ]:
# Univariate and multivariate boxplots for numeric columns
fig, ax =plt.subplots(1,2)
sns.boxplot(x=df['arbevert_2014'], ax=ax[0])
sns.boxplot(x='KATEGORIKUS_VALTOZO_2014', y='arbevert_2014', data=df, ax=ax[1])
plt.show()

In [ ]:
# Univariate and multivariate boxplots for numeric columns
fig, ax =plt.subplots(1,2)
sns.boxplot(x=df['target_reg'], ax=ax[0])
sns.boxplot(x='KATEGORIKUS_VALTOZO_2015', y='target_reg', data=df, ax=ax[1])
plt.show()

In [ ]:
# Outlier detection using IsolationForest

https://towardsdatascience.com/anomaly-detection-with-isolation-forest-visualization-23cd75c281e2
https://scikit-learn.org/stable/auto_examples/index.html
https://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html#sphx-glr-auto-examples-preprocessing-plot-all-scaling-py

In [ ]:
# Outlier detection using IsolationForest

from sklearn.ensemble import IsolationForest

X = np.array([[-1, -1], [-2, -1], [-3, -2], [0, 0], [-20, 50], [3, 5]])
clf = IsolationForest(n_estimators=10, warm_start=True)
clf.fit(X)  # fit 10 trees  
clf.set_params(n_estimators=20)  # add 10 more trees  
clf.fit(X)  # 

# Create model df

In [ ]:
# model columns
model_columns = ['transformed']+ ['KATEGORIKUS_VALTOZO_2014'] + ['KATEGORIKUS_VALTOZO_2015']+ ['KATEGORIKUS_VALTOZO_2016']+ 
                numeric_cols_2014 + numeric_cols_2015 + numeric_cols_2016 + columns_obj_perc

df_model = df[model_columns]

In [ ]:
# quick check for model columns
print('NR of features :: {}'.format(len(df_model.columns)))

df_model.sample(3)

# Correlation

In [ ]:
corrs = df.corr()
corrs

In [ ]:
filteredDf = dfCorr[((dfCorr >= .5) | (dfCorr <= -.5)) & (dfCorr !=1.000)]
filteredDf

In [ ]:
# Heatmap of the correlations
plt.figure(figsize=(10,8))
sns.heatmap(corrs, cmap='RdBu_r', annot=True)
plt.show()

In [ ]:
# create df for correlation
dfCorr = df.corr()
filteredDf = dfCorr[((dfCorr >= .7) | (dfCorr <= -.7)) & (dfCorr !=1.000)]

In [ ]:
# visualise correlation
plt.figure(figsize=(30,10))
sns.heatmap(filteredDf, annot=True, cmap="Reds")
plt.show()

In [ ]:
dfCorr = df.corr().unstack().sort_values().drop_duplicates()
dfCorr[((dfCorr >= .8) | (dfCorr <= -.8)) & (dfCorr !=1.000)]

In [ ]:
df_2015.sample(1)
df_2016.columns

In [ ]:
# correlation for 2016
dfCorr = df_2016.corr().unstack().sort_values().drop_duplicates()
dfCorr[((dfCorr >= .8) | (dfCorr <= -.8)) & (dfCorr !=1.000)]

# xgboost basic model 1

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold

import xgboost as xgb
print(xgb.__version__)

In [ ]:
# create an xgboost basic regression model
model = xgb.XGBRegressor()

In [ ]:
# create an xgboost regression model by configuring hyperparameters 
model = xgb.XGBRegressor(n_estimators=1000, max_depth=7, eta=0.1, subsample=1, colsample_bytree=1)

In [ ]:
df_2016.columns
df.select_dtypes(include='float64')

In [ ]:
# split data into input and output columns
X, y = df_2016.iloc[:, 2:len(df_2016)], df_2016.iloc[:, 1]
X = X.select_dtypes(include='float64')


In [ ]:
len(X)

In [ ]:
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force scores to be positive
scores = np.absolute(scores)
print('Mean MAE: %.3f (%.3f)' % (scores.mean(), scores.std()) )

# xgboost basic model 2

## Exhaustively search for the optimal hyperparameters

In [ ]:
import xgboost as xgb
print(xgb.__version__)

#=========================================================================
# XGBoost regression: 
# Parameters: 
# n_estimators  "Number of gradient boosted trees. Equivalent to number 
#                of boosting rounds."
# learning_rate "Boosting learning rate (also known as “eta”)"
# max_depth     "Maximum depth of a tree. Increasing this value will make 
#                the model more complex and more likely to overfit." 
#=========================================================================
regressor=xgb.XGBRegressor(eval_metric='rmse')

#=========================================================================
# exhaustively search for the optimal hyperparameters
#=========================================================================
from sklearn.model_selection import GridSearchCV
# set up our search grid
param_grid = {"max_depth":    [4, 5,8],
              "n_estimators": [101, 207, 700],
              "learning_rate": [0.001, 0.01, 0.015]}

# try out every combination of the above values
search = GridSearchCV(regressor, param_grid, cv=5).fit(X, y)

print("The best hyperparameters are ",search.best_params_)

In [ ]:
# using the calculated parameters
from sklearn.model_selection import train_test_split

# train and test split
X, y = df_model.iloc[:, 2:len(df_model)], df_model.iloc[:, 1]

X_train,X_test,y_train,y_test = train_test_split(X,y, test_size = 0.2)

regressor=xgb.XGBRegressor(learning_rate = search.best_params_["learning_rate"],
                           n_estimators  = search.best_params_["n_estimators"],
                           max_depth     = search.best_params_["max_depth"],
                           eval_metric='rmse')

regressor.fit(X_train, y_train, verbose=True)

predictions = regressor.predict(X_test)

## Without GridSearch using test.csv

In [ ]:
import xgboost as xgb
# print(xgb.__version__)

regressor=xgb.XGBRegressor(eval_metric='rmse')

from sklearn.model_selection import train_test_split

# split data into input and output columns
X, y = df_model.iloc[:, 2:len(df_model)], df_model.iloc[:, 1]

#X_train,X_test,y_train,y_test = train_test_split(X,y, test_size = 0.2)

regressor=xgb.XGBRegressor(objective ='reg:squarederror',
                          learning_rate = 0.001,
                           n_estimators  = 100,
                           max_depth     = 35,
                           alpha = 10,
                           eval_metric='rmse')

regressor.fit(X, y, verbose=True)

predictions = regressor.predict(df_test)


In [ ]:
# calculate MAE, MSE - not having target_reg trz using arbevetexp_2016 as the most closeset data
from sklearn.metrics import mean_squared_error, mean_absolute_error

mae = np.sqrt(mean_absolute_error(df_test['arbevetexp_2016'], predictions))
print("MAE: %f" % (mae))
mse = np.sqrt(mean_squared_error(df_test['arbevetexp_2016'], predictions))
print("MSE: %f" % (mse))


In [ ]:
predictions[200:203]

In [ ]:
df_pred= pd.DataFrame(predictions)
df_test['prediction'] = df_pred

df_test['inv_pred'] = pd.DataFrame(pt.inverse_transform(df_pred.values.reshape(-1, 1)))
df_test['inv_pred'] = df_test.inv_pred*df_test_orig.letszam_2016

df_test['arbevexp_inv'] = pd.DataFrame(pt.inverse_transform(df_test.arbevexp_2016.values.reshape(-1, 1)))
df_test['arbevexp_inv'] = df_test.arbevexp_inv*df_test_orig.letszam_2016

In [ ]:
df_test[['prediction','inv_pred', 'arbevexp_2016', 'arbevexp_inv']][200:203]

In [ ]:
df_test_orig[['arbevexp_2016', 'letszam_2016']][200:203]

## With GridSearch by 80-20 rule

In [ ]:
import xgboost as xgb
#print(xgb.__version__)

regressor=xgb.XGBRegressor(eval_metric='rmse')

from sklearn.model_selection import train_test_split

# split data into input and output columns
X, y = df_model.iloc[:, 2:len(df_model)], df_model.iloc[:, 1]

# split data into train and test
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size = 0.2)

regressor=xgb.XGBRegressor(objective ='reg:squarederror',
                          learning_rate = 0.001,
                           n_estimators  = 100,
                           max_depth     = 35,
                           alpha = 10,
                           eval_metric='rmse')

regressor.fit(X_train, y_train, verbose=True)

predictions = regressor.predict(X_test)

In [ ]:
# calculate MAE, MSE
from sklearn.metrics import mean_squared_error, mean_absolute_error

mae = np.sqrt(mean_absolute_error(y_test, predictions))
print("MAE: %f" % (mae))
mse = np.sqrt(mean_squared_error(y_test, predictions))
print("MSE: %f" % (mse))


In [ ]:
predictions[11:15]

In [ ]:
y_test[11:15]

In [ ]:
df_pred= pd.DataFrame(predictions)
df['prediction'] = df_pred
df['inv_pred'] = pd.DataFrame(ptt.inverse_transform(df_pred.values.reshape(-1, 1)))
#df['inv_pred'] = df.inv_pred*df.letszam_2016

In [ ]:
df_ = df.dropna()

In [ ]:
# calculate MAE, MSE
from sklearn.metrics import mean_squared_error, mean_absolute_error

mae = np.sqrt(mean_absolute_error(df_['target_reg'], df_['inv_pred']))
print("MAE: %f" % (mae))
mse = np.sqrt(mean_squared_error(df_['target_reg'], df_['inv_pred']))
print("MSE: %f" % (mse))

In [ ]:
df['inverse'] = pd.DataFrame(ptt.inverse_transform(df['transformed'].values.reshape(-1, 1)))
#df['inverse'] = df.inverse*df.letszam_2016
df['arbevexp_inv'] = pd.DataFrame(pt.inverse_transform(df['arbevexp_2016'].values.reshape(-1, 1)))
#df['arbevexp_inv'] = df.arbevexp_inv*df.letszam_2016


In [ ]:
df[[ 'target_reg', 'transformed', 'inverse','prediction', 'inv_pred', 'arbevexp_2016', 'arbevexp_inv', 'letszam_2016']][11:15]

## Visualise tree 

In [ ]:
data_dmatrix = xgb.DMatrix(data=X,label=y)

In [ ]:
xg_reg = xgb.train(params=params, dtrain=data_dmatrix, num_boost_round=10)

# Miscellous

In [ ]:
df[['arbevert_2014', 'arbevert_2015', 'arbevert_2016']].hist(figsize=(14,14), xrot=45)

In [ ]:
# for 'alakul_ev' use the most frequent values 
from sklearn.impute import SimpleImputer

imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
df['alakul_ev'] = imp.fit_transform(df['alakul_ev'].values.reshape(-1, 1))

In [ ]:
# aggregation, multi-index

df.groupby("quality").agg({
    "fixed acidity": ["sum", "mean", "std"],
    "volatile acidity": ["sum", "mean", "std"]
})

#You will notice that the column names have been put on two levels. This is called a MultiIndex and works a little differently. This is looked at in more detail in another guide, but here are a couple of quick tips:

df["fixed acidity"] # will now return a DataFrame with the three columns under fixed acidity.
df[("fixed acidity", "sum")] # will select the specific fixed acidity sum column.

#If you just want to flatten the index into one level, you can do the following:

df_grouped.columns = df_grouped.columns.to_flat_index()
df_grouped

#  drop one level index
df_grouped.droplevel(0)

In [ ]:
# You can use the following syntax to calculate the percentage of a total within groups in pandas:
df['values_var'] / df.groupby('group_var')['values_var'].transform('sum')


In [ ]:
# misc dealing with column names
cols_no_year = ['id', 'target_reg', 'alakul_ev']

columns_2014_obj = [column for column in df.select_dtypes(include='object').columns if "2014" in column]

columns_2015 = [column for column in df.columns if "2015" in column]
columns_2016 = [column for column in df.columns if "2016" in column]

columns = cols_no_year + columns_2014
columns_2014_obj.remove(['GAZDALKODASI_FORMA_2014_1_alkategoria'])

# if "2014" in column
obj = [column for column in df.select_dtypes( include=['object']).columns if "2014" in column]
obj
num = [column for column in df.select_dtypes(exclude=['int64'], include=['float64']).columns if "2014" in column]
#num = num.remove(['KATEGORIKUS_VALTOZO_2014', 'letszam_2014'])
num = [item for item in num if item not in  ['KATEGORIKUS_VALTOZO_2014', 'letszam_2014']]
num
 

In [ ]:
# preprocess categorical columns - transform them in percentage ones
df_ = df.copy()

for col in columns_2014_obj:
    # calculate the percentage 
    df_grouped = df_[col].value_counts(normalize = True)
    # get the index as column
    df_grouped = df_grouped.reset_index()
    # change the column names :: orig column and percent
    percent_col = '%'+col
    df_grouped.columns = [col, percent_col]
    # merge the percent values to orig df
    df_ = pd.merge(df_, df_grouped, how='left')

In [ ]:
# Add percentage column - 1 column
df_ = df.copy()

col = 'MEGYE_2014'

# calculate the percentage 
df_grouped = df_[col].value_counts(normalize = True)
# get the index as column
df_grouped = df_grouped.reset_index()
# change the column names :: orig column and percent
percent_col = '%'+col
df_grouped.columns = [col, percent_col]
# merge the percent values to orig df
df_ = pd.merge(df_, df_grouped, how='left')

In [ ]:
# remove alkategoriak
to_be_removed_list = ['GAZDALKODASI_FORMA_2014_1_alkategoria', 'GAZDALKODASI_FORMA_2014_2_alkategoria', 
                      'KKV_BESOROLAS_2014_1_alkategoria','KKV_BESOROLAS_2014_2_alkategoria',
                      'agazat_2014_1_alkategoria','agazat_2014_2_alkategoria','agazat_2014_3_alkategoria',
                      
                      'GAZDALKODASI_FORMA_2015_1_alkategoria', 'GAZDALKODASI_FORMA_2015_2_alkategoria', 
                      'KKV_BESOROLAS_2015_1_alkategoria','KKV_BESOROLAS_2015_2_alkategoria',
                      'agazat_2015_1_alkategoria','agazat_2015_2_alkategoria','agazat_2015_3_alkategoria',
                      
                      'GAZDALKODASI_FORMA_2016_1_alkategoria', 'GAZDALKODASI_FORMA_2016_2_alkategoria', 
                      'KKV_BESOROLAS_2016_1_alkategoria','KKV_BESOROLAS_2016_2_alkategoria',
                      'agazat_2016_1_alkategoria','agazat_2016_2_alkategoria','agazat_2016_3_alkategoria'
                     ]

columns_obj = [item for item in columns_obj_all if item not in to_be_removed_list]
print("Required columns:: \n{}".format( columns_obj))

In [ ]:
# np.log(ertek_2016)
df['log_ertek_2016'] = np.log(df['ertek_2016'])

plt.figure()
sns.distplot(df['log_ertek_2016'], hist=False, kde=True)
plt.show()

In [ ]:
# Finding the outliers

def calculateIQR(x):
    q1 = df.quantile(0.25)
    q3 = df.quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 -(1.5 * iqr) 
    upper_bound = q3 +(1.5 * iqr)
    return iqr, lower_bound, upper_bound

#calculate IQR for all columns
#result = df[columns].apply(calculateIQR)
#print('\nThe Interquartile Range for all columns are as follows:\n',result)

In [ ]:
# transform target_reg with box-cox
from scipy.stats import boxcox 
from scipy.stats import boxcox, zscore
from scipy.stats.mstats import winsorize

# set 0s to 0.1 for boxcox to work
df['target_reg_'] = np.where(df.target_reg>0, df.target_reg,0.01)
print(len(df[df.target_reg_ ==0]))

#perform Box-Cox transformation on original data
df['transformed'], best_lambda = boxcox(df['target_reg_']) 
print("Best_lambda {}".format(best_lambda))

# Histogram and kernel density estimate
plt.figure()
sns.distplot(df['transformed'], hist=False, kde=True)
plt.show()

In [ ]:
# quick check
columns = ['target_reg','transformed']
df[columns].describe(include='float64')

In [ ]:
# dfs per years
def year_columns(year):
   year_list = 'columns' + year
   year_list = [column for column in df.columns if year in column]
   return year_list

df_2014 = df[cols_no_year + year_columns('2014')]
df_2015 = df[cols_no_year + year_columns('2015')]
df_2016 = df[cols_no_year + year_columns('2016')]